# Desenvolvimento

In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('Datasets/trainset.csv')

In [18]:
df.head()

,Unnamed: 0,srcip,srcport,dstip,dstport,proto,total_fpackets,total_fvolume,total_bpackets,total_bvolume,...,total_packets,total_bits,bpp,bps,pps,var_iat,avg_iat,pct_packets_pushed,avg_payload_length,iopr
0,0,147.32.84.180,1027,74.125.232.195,80,6,7,1082,3,205,...,10,10296,128.700000,0.000000e+00,0.000000e+00,0.0,0.0,0.700000,86.300000,0.428571
1,1,147.32.84.180,1040,94.63.149.152,80,6,32,1476,11,26238,...,43,221712,644.511628,0.000000e+00,0.000000e+00,0.0,0.0,0.744186,603.953488,0.343750
2,2,147.32.84.180,1041,60.190.223.75,2012,6,60,2618,28,57221,...,88,478712,679.988636,3.989267e+10,7.333333e+06,0.0,0.0,0.681818,639.715909,0.466667
3,3,147.32.84.180,1044,60.190.223.75,888,6,9,1006,3,1882,...,12,23104,240.666667,3.850667e+09,2.000000e+06,0.0,0.0,0.750000,198.666667,0.333333
4,4,147.32.84.180,1046,147.32.84.171,139,6,21,2906,11,1364,...,32,34160,133.437500,0.000000e+00,0.000000e+00,0.0,0.0,0.656250,92.687500,0.523810


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12758 entries, 0 to 12757
Data columns (total 58 columns):
Unnamed: 0            12758 non-null int64
srcip                 12758 non-null object
srcport               12758 non-null int64
dstip                 12758 non-null object
dstport               12758 non-null int64
proto                 12758 non-null int64
total_fpackets        12758 non-null int64
total_fvolume         12758 non-null int64
total_bpackets        12758 non-null int64
total_bvolume         12758 non-null int64
min_fpktl             12758 non-null int64
mean_fpktl            12758 non-null int64
max_fpktl             12758 non-null int64
std_fpktl             12758 non-null int64
min_bpktl             12758 non-null int64
mean_bpktl            12758 non-null int64
max_bpktl             12758 non-null int64
std_bpktl             12758 non-null int64
min_fiat              12758 non-null int64
mean_fiat             12758 non-null int64
max_fiat              12758 n

O dataset acima ja contem todas as características adicionais criadas manualmente assim como todas as antigas geradas por ele. Para o conjunto de treino, existem algumas características que ja podem ser eliminadas:

- Ip/port de origem e destino: características importante na hora da geração de fluxo e rotulação, não possuem muita utilidade nas tarefas de classificação;
- Protocolo: como todos os fluxos restantes são do protocolo TCP, essa coluna se torna irrelevante pois só apresenta um valor para todas as linhas;
- Além disso, é possível eliminar todas as colunas que somente apresentam valores de 0, no caso: std_active, min_idle,mean_idle,max_idle,std_idle,furg_cnt,burg_cnt.
- Por fim, é possível eliminar as colunas que contém as características sobre subflows, que não são relevantes para o projeto: 'sflow_bpackets','sflow_bbytes','sflow_fpackets','sflow_fbytes'.

In [21]:
df.drop(['Unnamed: 0','srcip', 'srcport', 'dstip', 'dstport', 'proto', 'std_active', 'min_idle', 
        'mean_idle', 'max_idle', 'std_idle','furg_cnt', 'burg_cnt','sflow_bpackets','sflow_bbytes',
        'sflow_fpackets','sflow_fbytes','dscp'],axis=1,inplace=True)

In [22]:
df.columns

Index(['total_fpackets', 'total_fvolume', 'total_bpackets', 'total_bvolume',
       'min_fpktl', 'mean_fpktl', 'max_fpktl', 'std_fpktl', 'min_bpktl',
       'mean_bpktl', 'max_bpktl', 'std_bpktl', 'min_fiat', 'mean_fiat',
       'max_fiat', 'std_fiat', 'min_biat', 'mean_biat', 'max_biat', 'std_biat',
       'duration', 'min_active', 'mean_active', 'max_active', 'fpsh_cnt',
       'bpsh_cnt', 'total_fhlen', 'total_bhlen', 'label', 'total_bytes',
       'total_packets', 'total_bits', 'bpp', 'bps', 'pps', 'var_iat',
       'avg_iat', 'pct_packets_pushed', 'avg_payload_length', 'iopr'],
      dtype='object')

In [24]:
len(df.columns)

40

Sem contar a coluna de rótulos, o processo acima nos deixa ainda com 39 características. 

A partir de agora, o processo de desenvolvimento se dará através dos seguintes passos:

1. Separação do conjunto de hold-out, ou seja, um subconjunto dos dados de treinamento utilizados para verificar o desempenho geral dos classificadores;
2. Seleção dos algoritmos a serem utilizados;
3. Seleção das características;
4. Verificação dos resultados;
5. Aplicação dos algoritmos no conjunto de teste.

# Hold-out

o Scikit learn possui uma função chamada train_test_split, que separa o conjunto passado entre o conjunto de treino e o conjunto de teste (hold-out) de maneira randômica

Um exemplo de uso básico dessa função presente na documentação pode ser visto a seguir:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Parâmetros:

X: conjunto de características
y: conjunto de rótulos
test_size: define o tamanho do conjunto de treino. No caso, é estabelecida uma porcentagem de 33% para ele
random_state: seed utilizada pelo gerador randômico para a geração

Para o caso deste trabalho, será estabelecido uma divisão tradicional de 70% para conjunto de treino e 30% para o conjunto de teste.

In [27]:
from sklearn.model_selection import train_test_split

In [39]:
X = df.drop(['label'],axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Seleção dos algoritmos a serem utilizados

Para decidir os algoritmos que seriam utilizados, foram analisados dados presentes no paper Machine learning for identifying botnet network traffic, que faz um levantamento sobre vários estudos da área de Machine Learning para Detecção de botnets e suas principais características e diferenças. Ele faz um estudo sobre se os estudos utilizam métodos supervisionados ou não supervisionados, que tipo de protocolo buscam identificar, entre outros.

Filtrando as características para estudos de botnets IRC baseados em métodos de Machine Learning supervisionado, é possível verificar os seguintes estimadores: **Árvores de decisão, Support Vector Machines, Naive Bayes, Redes Bayesianas, e Boosted Decision Trees.**

Dos citados acima, Redes bayesianas é o único que não será implementado por não possuir implementação no Scikit. Além disso, será testado também o classificadores de **Florestas Aleátorias**. Já para Boosted Decision Trees, serão utilizados os algoritmo **AdaBoost**.

In [26]:
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB

# Seleção de características

Serão feitas duas abordagens para seleção de características:

1. **Seleção com base em pesquisas anteriores**: serão utilizadas somente as características presentes em trabalhos da área, e através de uma pesquisa de força bruta rodar todas as combinações delas com todos os algoritmos e assim desta maneira encontrar o melhor conjunto de características para cada classificador.


2. **Seleção utilizando algoritmos específicos**: o scikit possui vários algoritmos para seleção de características, que buscam encontrar dado um conjunto a melhor combinação. Para este trabalho, será utilizado o RFE (Recursive Feature Elimination) no conjunto inteiro de dados, tanto os do gerador de fluxo quanto os criados com base em estudos anteriores. A ideia desta abordagem é verificar quão eficar são as features utilizadas historicamente na área, e também buscar identificar se algumas das fornecidas pelo gerador são eficazes nesse processo

# Primeira abordagem: Características de trabalhos anteriores

In [14]:
from time import time
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix,accuracy_score
from sklearn.model_selection import cross_val_predict

In [ ]:
import itertools
# Lista que guardará todas as combinações
feat_sets = []

#Todas as colunas utilizadas
nomes_colunas = ['duration','total_packets','iopr','total_bytes','avg_payload_length','bps','bpp','pps',
            'pct_packets_pushed','var_iat','avg_iat','mean_active']

# Variando de 4 até o número máximo de características
for T in range(4, len(all_cols)+1):
    # iteração de todos os subsets de tamanho T
    for subset in itertools.combinations(nomes_colunas, T):
        array_nomes = [nome for nome in subset]
        feat_sets.append(array_nomes)

In [16]:
len(feat_sets)

3797

In [17]:
classifiers = [DecisionTreeClassifier(),MultinomialNB(),GaussianNB(),BernoulliNB(),LinearSVC(),AdaBoostClassifier(),RandomForestClassifier()]
names = ['Decision Tree','Multinominal NB','Gaussian NB','Bernoulli NB','Linear SVC','Adaboost','Random Forest Classifier']

results = []

for i,subset in enumerate(feat_sets):
    print('Running subset {}'.format(str(subset)))
    X = df[subset]
    y = df['label']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)
    
    for j, c in enumerate(classifiers):
        start = time()
        print('Classifier {}'.format(names[j]))
        predictions = c.fit(X_train,y_train).predict(X_test)
        precision,recall,fscore,support = precision_recall_fscore_support(y_test,predictions)
        acc = accuracy_score(y_test,predictions)
        print('Accuracy: {}'.format(acc))
        c_matrix = confusion_matrix(y_test,predictions)
        results.append([i,names[j],acc,precision,recall,fscore,c_matrix,(time() - start)])

Running subset ['duration', 'total_packets', 'iopr', 'total_bytes']
Classifier Decision Tree
Accuracy: 0.9947753396029259
Classifier Multinominal NB
Accuracy: 0.4231974921630094
Classifier Gaussian NB
Accuracy: 0.5462382445141066
Classifier Bernoulli NB
Accuracy: 0.6230407523510971
Classifier Linear SVC
Accuracy: 0.7392894461859979
Classifier Adaboost
Accuracy: 0.9845872518286312
Classifier Random Forest Classifier
Accuracy: 0.9952978056426333
Running subset ['duration', 'total_packets', 'iopr', 'avg_payload_length']
Classifier Decision Tree
Accuracy: 0.9950365726227796
Classifier Multinominal NB
Accuracy: 0.5835945663531871
Classifier Gaussian NB
Accuracy: 0.8215778474399164
Classifier Bernoulli NB
Accuracy: 0.6230407523510971
Classifier Linear SVC
Accuracy: 0.8649425287356322
Classifier Adaboost
Accuracy: 0.9882445141065831
Classifier Random Forest Classifier
Accuracy: 0.9950365726227796
Running subset ['duration', 'total_packets', 'iopr', 'bps']
Classifier Decision Tree
Accuracy: 0.

Accuracy: 0.6752873563218391
Classifier Adaboost
Accuracy: 0.9796238244514106
Classifier Random Forest Classifier
Accuracy: 0.9853709508881923
Running subset ['duration', 'total_packets', 'avg_payload_length', 'pps']
Classifier Decision Tree
Accuracy: 0.9840647857889238
Classifier Multinominal NB
Accuracy: 0.7552246603970741
Classifier Gaussian NB
Accuracy: 0.5310867293625914
Classifier Bernoulli NB
Accuracy: 0.6230407523510971
Classifier Linear SVC
Accuracy: 0.41849529780564265
Classifier Adaboost
Accuracy: 0.98380355276907
Classifier Random Forest Classifier
Accuracy: 0.9866771159874608
Running subset ['duration', 'total_packets', 'avg_payload_length', 'pct_packets_pushed']
Classifier Decision Tree
Accuracy: 0.9942528735632183
Classifier Multinominal NB
Accuracy: 0.5527690700104493
Classifier Gaussian NB
Accuracy: 0.5660919540229885
Classifier Bernoulli NB
Accuracy: 0.6230407523510971
Classifier Linear SVC
Accuracy: 0.5104493207941484
Classifier Adaboost
Accuracy: 0.9882445141065831


Accuracy: 0.5540752351097179
Classifier Adaboost
Accuracy: 0.9691745036572623
Classifier Random Forest Classifier
Accuracy: 0.9733542319749217
Running subset ['duration', 'total_packets', 'pps', 'mean_active']
Classifier Decision Tree
Accuracy: 0.9670846394984326
Classifier Multinominal NB
Accuracy: 0.7701149425287356
Classifier Gaussian NB
Accuracy: 0.5104493207941484
Classifier Bernoulli NB
Accuracy: 0.6230407523510971
Classifier Linear SVC
Accuracy: 0.4422675026123302
Classifier Adaboost
Accuracy: 0.9660397074190178
Classifier Random Forest Classifier
Accuracy: 0.9689132706374086
Running subset ['duration', 'total_packets', 'pct_packets_pushed', 'var_iat']
Classifier Decision Tree
Accuracy: 0.9952978056426333
Classifier Multinominal NB
Accuracy: 0.4939916405433647
Classifier Gaussian NB
Accuracy: 0.49164054336468127
Classifier Bernoulli NB
Accuracy: 0.8202716823406478
Classifier Linear SVC
Accuracy: 0.5559038662486938
Classifier Adaboost
Accuracy: 0.9835423197492164
Classifier Rando

Accuracy: 0.5914315569487983
Classifier Adaboost
Accuracy: 0.9840647857889238
Classifier Random Forest Classifier
Accuracy: 0.9939916405433646
Running subset ['duration', 'iopr', 'avg_payload_length', 'var_iat']
Classifier Decision Tree
Accuracy: 0.9937304075235109
Classifier Multinominal NB
Accuracy: 0.49033437826541276
Classifier Gaussian NB
Accuracy: 0.491901776384535
Classifier Bernoulli NB
Accuracy: 0.8202716823406478
Classifier Linear SVC
Accuracy: 0.9310344827586207
Classifier Adaboost
Accuracy: 0.9858934169278997
Classifier Random Forest Classifier
Accuracy: 0.9934691745036572
Running subset ['duration', 'iopr', 'avg_payload_length', 'avg_iat']
Classifier Decision Tree
Accuracy: 0.9937304075235109
Classifier Multinominal NB
Accuracy: 0.49582027168234066
Classifier Gaussian NB
Accuracy: 0.7262277951933125
Classifier Bernoulli NB
Accuracy: 0.8390804597701149
Classifier Linear SVC
Accuracy: 0.7860501567398119
Classifier Adaboost
Accuracy: 0.9892894461859979
Classifier Random Fores

/home/luis/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

Accuracy: 0.3766980146290491
Classifier Adaboost
Accuracy: 0.991901776384535
Classifier Random Forest Classifier
Accuracy: 0.9942528735632183
Running subset ['duration', 'total_packets', 'iopr', 'total_bytes', 'avg_payload_length', 'bps', 'bpp', 'pct_packets_pushed', 'var_iat', 'avg_iat']
Classifier Decision Tree
Accuracy: 0.9916405433646813
Classifier Multinominal NB
Accuracy: 0.7774294670846394
Classifier Gaussian NB
Accuracy: 0.5318704284221526
Classifier Bernoulli NB
Accuracy: 0.8401253918495298
Classifier Linear SVC
Accuracy: 0.30276907001044934
Classifier Adaboost
Accuracy: 0.990073145245559
Classifier Random Forest Classifier
Accuracy: 0.9945141065830722
Running subset ['duration', 'total_packets', 'iopr', 'total_bytes', 'avg_payload_length', 'bps', 'bpp', 'pct_packets_pushed', 'var_iat', 'mean_active']
Classifier Decision Tree
Accuracy: 0.9921630094043887
Classifier Multinominal NB
Accuracy: 0.777951933124347
Classifier Gaussian NB
Accuracy: 0.5318704284221526
Classifier Bernou

Accuracy: 0.819226750261233
Classifier Linear SVC
Accuracy: 0.5120167189132706
Classifier Adaboost
Accuracy: 0.9916405433646813
Classifier Random Forest Classifier
Accuracy: 0.9950365726227796
Running subset ['duration', 'total_packets', 'iopr', 'total_bytes', 'bps', 'bpp', 'pps', 'pct_packets_pushed', 'var_iat', 'mean_active']
Classifier Decision Tree
Accuracy: 0.9913793103448276
Classifier Multinominal NB
Accuracy: 0.6891327063740856
Classifier Gaussian NB
Accuracy: 0.7082027168234065
Classifier Bernoulli NB
Accuracy: 0.8202716823406478
Classifier Linear SVC
Accuracy: 0.416405433646813
Classifier Adaboost
Accuracy: 0.9895506792058516
Classifier Random Forest Classifier
Accuracy: 0.995559038662487
Running subset ['duration', 'total_packets', 'iopr', 'total_bytes', 'bps', 'bpp', 'pps', 'pct_packets_pushed', 'avg_iat', 'mean_active']
Classifier Decision Tree
Accuracy: 0.9908568443051202
Classifier Multinominal NB
Accuracy: 0.6933124346917451
Classifier Gaussian NB
Accuracy: 0.7149947753

Accuracy: 0.3390804597701149
Classifier Adaboost
Accuracy: 0.9898119122257053
Classifier Random Forest Classifier
Accuracy: 0.9942528735632183
Running subset ['duration', 'total_packets', 'total_bytes', 'avg_payload_length', 'bps', 'bpp', 'pps', 'pct_packets_pushed', 'avg_iat', 'mean_active']
Classifier Decision Tree
Accuracy: 0.9924242424242424
Classifier Multinominal NB
Accuracy: 0.6959247648902821
Classifier Gaussian NB
Accuracy: 0.7149947753396029
Classifier Bernoulli NB
Accuracy: 0.8390804597701149
Classifier Linear SVC
Accuracy: 0.5929989550679206
Classifier Adaboost
Accuracy: 0.991901776384535
Classifier Random Forest Classifier
Accuracy: 0.9939916405433646
Running subset ['duration', 'total_packets', 'total_bytes', 'avg_payload_length', 'bps', 'bpp', 'pps', 'var_iat', 'avg_iat', 'mean_active']
Classifier Decision Tree
Accuracy: 0.9851097178683386
Classifier Multinominal NB
Accuracy: 0.6893939393939394
Classifier Gaussian NB
Accuracy: 0.7082027168234065
Classifier Bernoulli NB
A

Accuracy: 0.44592476489028215
Classifier Adaboost
Accuracy: 0.9892894461859979
Classifier Random Forest Classifier
Accuracy: 0.9950365726227796
Running subset ['duration', 'total_bytes', 'avg_payload_length', 'bps', 'bpp', 'pps', 'pct_packets_pushed', 'var_iat', 'avg_iat', 'mean_active']
Classifier Decision Tree
Accuracy: 0.9947753396029259
Classifier Multinominal NB
Accuracy: 0.6896551724137931
Classifier Gaussian NB
Accuracy: 0.7082027168234065
Classifier Bernoulli NB
Accuracy: 0.819226750261233
Classifier Linear SVC
Accuracy: 0.36859979101358414
Classifier Adaboost
Accuracy: 0.9898119122257053
Classifier Random Forest Classifier
Accuracy: 0.9942528735632183
Running subset ['total_packets', 'iopr', 'total_bytes', 'avg_payload_length', 'bps', 'bpp', 'pps', 'pct_packets_pushed', 'var_iat', 'avg_iat']
Classifier Decision Tree
Accuracy: 0.9934691745036572
Classifier Multinominal NB
Accuracy: 0.6880877742946708
Classifier Gaussian NB
Accuracy: 0.7082027168234065
Classifier Bernoulli NB
Ac

Accuracy: 0.819226750261233
Classifier Linear SVC
Accuracy: 0.5128004179728317
Classifier Adaboost
Accuracy: 0.9926854754440961
Classifier Random Forest Classifier
Accuracy: 0.9939916405433646
Running subset ['duration', 'total_packets', 'iopr', 'total_bytes', 'avg_payload_length', 'bps', 'bpp', 'pct_packets_pushed', 'var_iat', 'avg_iat', 'mean_active']
Classifier Decision Tree
Accuracy: 0.991901776384535
Classifier Multinominal NB
Accuracy: 0.777951933124347
Classifier Gaussian NB
Accuracy: 0.5318704284221526
Classifier Bernoulli NB
Accuracy: 0.819226750261233
Classifier Linear SVC
Accuracy: 0.3014629049111808
Classifier Adaboost
Accuracy: 0.990073145245559
Classifier Random Forest Classifier
Accuracy: 0.9947753396029259
Running subset ['duration', 'total_packets', 'iopr', 'total_bytes', 'avg_payload_length', 'bps', 'pps', 'pct_packets_pushed', 'var_iat', 'avg_iat', 'mean_active']
Classifier Decision Tree
Accuracy: 0.991901776384535
Classifier Multinominal NB
Accuracy: 0.6891327063740

In [18]:
results_df = pd.DataFrame(results,columns = ['Subset','Classifier','Accuracy','Precision','Recall','F-score','Confusion Matrix','Runtime'])

Obtendo subconjuntos de cada classificador, e verificando os conjuntos que obtiveram acurácia máxima para cada um. Em caso de empate, foi considerado o modelo com mais características.

In [20]:
rfstats = results_df[results_df['Classifier'] == 'Random Forest Classifier']
adastats = results_df[results_df['Classifier'] == 'Adaboost']
dtstats = results_df[results_df['Classifier'] == 'Decision Tree']
mnbstats = results_df[results_df['Classifier'] == 'Multinominal NB']
gnbstats = results_df[results_df['Classifier'] == 'Gaussian NB']
bnbstats = results_df[results_df['Classifier'] == 'Bernoulli NB']
svcstats = results_df[results_df['Classifier'] == 'Linear SVC']


In [21]:
dtstats[dtstats['Accuracy'] == dtstats['Accuracy'].max()]

,Subset,Classifier,Accuracy,Precision,Recall,F-score,Confusion Matrix,Runtime
3500,500,Decision Tree,0.996604,"[0.9962566844919786, 0.9969356486210419]","[0.9967897271268058, 0.9964267483409903]","[0.9965231345279486, 0.9966811335205514]","[[1863, 6], [7, 1952]]",0.026582
10857,1551,Decision Tree,0.996604,"[0.9967880085653105, 0.9964285714285714]","[0.9962546816479401, 0.996937212863706]","[0.9965212737489965, 0.9966828272518499]","[[1862, 7], [6, 1953]]",0.026630


In [28]:
feat_sets[1551]

['duration',
 'iopr',
 'total_bytes',
 'pct_packets_pushed',
 'avg_iat',
 'mean_active']

In [22]:
rfstats[rfstats['Accuracy'] == rfstats['Accuracy'].max()]

,Subset,Classifier,Accuracy,Precision,Recall,F-score,Confusion Matrix,Runtime
5991,855,Random Forest Classifier,0.99791,"[0.998393144081414, 0.9974502804691484]","[0.9973247726056714, 0.998468606431853]","[0.9978586723768736, 0.9979591836734695]","[[1864, 5], [3, 1956]]",0.081676


In [29]:
feat_sets[855]

['total_packets', 'iopr', 'avg_payload_length', 'bps', 'pct_packets_pushed']

In [23]:
adastats[adastats['Accuracy'] == adastats['Accuracy'].max()]

,Subset,Classifier,Accuracy,Precision,Recall,F-score,Confusion Matrix,Runtime
22657,3236,Adaboost,0.993208,"[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[[1856, 13], [13, 1946]]",0.771372
23189,3312,Adaboost,0.993208,"[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[[1856, 13], [13, 1946]]",0.658499
24302,3471,Adaboost,0.993208,"[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[[1856, 13], [13, 1946]]",0.732537
24470,3495,Adaboost,0.993208,"[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[[1856, 13], [13, 1946]]",0.733932
25436,3633,Adaboost,0.993208,"[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[[1856, 13], [13, 1946]]",0.979039
25457,3636,Adaboost,0.993208,"[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[[1856, 13], [13, 1946]]",0.816225
25625,3660,Adaboost,0.993208,"[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[[1856, 13], [13, 1946]]",0.805006
26003,3714,Adaboost,0.993208,"[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[[1856, 13], [13, 1946]]",0.921137
26388,3769,Adaboost,0.993208,"[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[0.9930444087747459, 0.9933639612046963]","[[1856, 13], [13, 1946]]",0.926338


In [30]:
feat_sets[3471]

['iopr',
 'total_bytes',
 'avg_payload_length',
 'bpp',
 'pps',
 'var_iat',
 'avg_iat',
 'mean_active']

In [24]:
mnbstats[mnbstats['Accuracy'] == mnbstats['Accuracy'].max()]

,Subset,Classifier,Accuracy,Precision,Recall,F-score,Confusion Matrix,Runtime
540,77,Multinominal NB,0.900209,"[0.9525258673158856, 0.8608695652173913]","[0.8373461744248261, 0.9601837672281777]","[0.8912300683371299, 0.9078185328185329]","[[1565, 304], [78, 1881]]",0.010539


In [31]:
feat_sets[77]

['duration', 'iopr', 'pct_packets_pushed', 'mean_active']

In [25]:
gnbstats[gnbstats['Accuracy'] == gnbstats['Accuracy'].max()]

,Subset,Classifier,Accuracy,Precision,Recall,F-score,Confusion Matrix,Runtime
2242,320,Gaussian NB,0.863114,"[0.8656878738444806, 0.8607340372046255]","[0.8517924023542001, 0.8739152628892292]","[0.8586839266450917, 0.8672745694022289]","[[1592, 277], [247, 1712]]",0.00927


In [32]:
feat_sets[320]

['iopr', 'avg_payload_length', 'bpp', 'pct_packets_pushed']

In [26]:
bnbstats[bnbstats['Accuracy'] == bnbstats['Accuracy'].max()]

,Subset,Classifier,Accuracy,Precision,Recall,F-score,Confusion Matrix,Runtime
297,42,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.013327
549,78,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.013018
745,106,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.016802
892,127,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.009192
997,142,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.011961
1067,152,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.012036
1109,158,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.009399
1130,161,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.009312
1151,164,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.010503
1389,198,Bernoulli NB,0.840125,"[0.8049490538573508, 0.8811544991511036]","[0.8876404494382022, 0.7947932618683001]","[0.8442748091603052, 0.8357487922705312]","[[1659, 210], [402, 1557]]",0.011115


In [33]:
feat_sets[3779]

['total_packets',
 'iopr',
 'total_bytes',
 'avg_payload_length',
 'bpp',
 'pps',
 'pct_packets_pushed',
 'var_iat',
 'avg_iat',
 'mean_active']

In [27]:
svcstats[svcstats['Accuracy'] == svcstats['Accuracy'].max()]

,Subset,Classifier,Accuracy,Precision,Recall,F-score,Confusion Matrix,Runtime
4806,686,Linear SVC,0.960293,"[0.9908519153802172, 0.9345839345839346]","[0.9272338148742643, 0.9918325676365493]","[0.9579878385848535, 0.9623576027736502]","[[1733, 136], [16, 1943]]",0.63157


In [34]:
feat_sets[686]

['duration', 'iopr', 'bpp', 'var_iat', 'avg_iat']

In [ ]:
results_df.to_csv('bruteforce_results.csv')

# Segunda abordagem: Utilizando RFE

Para este trabalho, será utilizado o RFECV, que implementa o algoritmo de Recursive Feature Elimination em conjunto da técnica de Cross Validation para identificar o número ótimo de features a ser utilizado. Ele possui a seguinte interface básica:

RFECV(estimator, step=1)

Parâmetros:

- estimator: o classificador a ser avaliado
- step: o número de passos a ser feito na quantidade de características a cada iteração.

Além destes parâmetros básicos, ainda é possível definir o tipo de cross validation e scoring a ser utilizado, assim como o número de threads e verbosidades da saída do algoritmo.

O processo então será rodar o RFECV com cada um dos classificadores estudados e verificar sua efieciencia.

In [13]:
from sklearn.metrics import accuracy_score

In [44]:
classifiers = [DecisionTreeClassifier(),MultinomialNB(),BernoulliNB(),LinearSVC(),AdaBoostClassifier(),RandomForestClassifier()]
names = ['Decision Tree','Multinominal NB','Bernoulli NB','Linear SVC','Adaboost','Random Forest Classifier']

for i,clf in enumerate(classifiers):
    
    print('Testing classifier {}'.format(names[i]))
    
    rfecv = RFECV(clf,verbose=3)
    
    rfecv.fit(X_train,y_train)
    
    print("Optimal number of features : %d" % rfecv.n_features_)
    
    rankings = list(rfecv.ranking_)
    best = []
    for index,content in enumerate(rankings):
        if(content == 1):
            best.append(all_cols[index])
            
    print('Optimal subset: ' + str(best))
    
    rf = clf

    Xaux_train = X_train[best]
    Xaux_test = X_test[best]

    rf.fit(Xaux_train,y_train)
    pred = rf.predict(Xaux_test)

    print(accuracy_score(y_test,pred))

Testing classifier Decision Tree
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 featu

Optimal number of features : 17
Optimal subset: ['total_fpackets', 'total_bpackets', 'min_fpktl', 'min_bpktl', 'min_fiat', 'mean_fiat', 'max_fiat', 'std_fiat', 'min_biat', 'mean_biat', 'max_biat', 'std_biat', 'fpsh_cnt', 'bpsh_cnt', 'avg_iat', 'pct_packets_pushed', 'iopr']
0.9503657262277951
Testing classifier Bernoulli NB
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 f

Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Optimal number of features : 12
Optimal subset: ['min_fpktl', 'mean_fiat', 'max_fiat', 'std_fiat', 'mean_biat', 'max_biat', 'std_biat', 'bpp', 'avg_iat', 'pct_packets_pushed', 'avg_payload_length', 'iopr']
0.9563740856844305
Testing classifier Adaboost
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.

Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.


## Otimização de Hiperparâmetros

### SVC

In [32]:
all_cols = ['min_fpktl', 'mean_fiat', 'max_fiat', 'std_fiat',
            'mean_biat', 'max_biat', 'std_biat', 'bpp', 'avg_iat', 'pct_packets_pushed',
            'avg_payload_length', 'iopr']
X = df[all_cols]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Kernel Linear

In [34]:
param_grid = {'C':[0.1,1,10,100]}

grid = GridSearchCV(SVC(),param_grid,verbose=3,n_jobs=2)
grid.fit(X_train,y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] .................. C=0.1, score=0.8504704301075269, total=   2.4s
[CV] C=0.1 ...........................................................
[CV] .................. C=0.1, score=0.8482202820685023, total=   2.7s
[CV] C=1 .............................................................
[CV] .................. C=0.1, score=0.8541666666666666, total=   1.9s
[CV] C=1 .............................................................
[CV] .................... C=1, score=0.9281396910678308, total=   2.1s
[CV] C=1 .............................................................
[CV] .................... C=1, score=0.9237231182795699, total=   2.7s
[CV] C=10 ............................................................
[CV] .................... C=1, score=0.9213709677419355, total=   2.6s
[CV] C=10 .......

[Parallel(n_jobs=2)]: Done  12 out of  12 | elapsed:   21.6s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'C': [0.1, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=3)

In [35]:
grid.best_params_

{'C': 100}

In [36]:
grid.best_estimator_

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
pred = grid.predict(X_test)
accuracy_score(y_test,pred)

0.9409613375130617

#### Kernel RBF 

In [29]:
from sklearn.model_selection import GridSearchCV

In [15]:
param_grid = {'C':[0.1,1,10,100]}

grid = GridSearchCV(SVC(),param_grid)
grid.fit(X_train,y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] C=0.1, gamma=1 ..................................................
[CV] ......... C=0.1, gamma=1, score=0.8067876344086021, total=   4.9s
[CV] C=0.1, gamma=1 ..................................................
[CV] ......... C=0.1, gamma=1, score=0.8143049026192075, total=   5.1s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ....... C=0.1, gamma=0.1, score=0.8408327736736064, total=   1.8s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ......... C=0.1, gamma=1, score=0.8175403225806451, total=   2.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ....... C=0.1, gamma=0.1, score=0.8484543010752689, total=   2.1s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ....... C=0.1, gamma=0.1, score=0.8497983870967742, total=   2.1s
[CV] C=0.1, gamm

[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:   31.4s


[CV] ...... C=1, gamma=0.0001, score=0.9633736559139785, total=   0.4s
[CV] C=10, gamma=1 ...................................................
[CV] ...... C=1, gamma=0.0001, score=0.9620295698924731, total=   0.4s
[CV] C=10, gamma=1 ...................................................
[CV] .......... C=10, gamma=1, score=0.8791134989926125, total=   2.0s
[CV] C=10, gamma=1 ...................................................
[CV] .......... C=10, gamma=1, score=0.8803763440860215, total=   2.0s
[CV] C=10, gamma=0.1 .................................................
[CV] ......... C=10, gamma=0.1, score=0.931497649429147, total=   2.1s
[CV] C=10, gamma=0.1 .................................................
[CV] .......... C=10, gamma=1, score=0.8823924731182796, total=   2.1s
[CV] C=10, gamma=0.1 .................................................
[CV] ........ C=10, gamma=0.1, score=0.9304435483870968, total=   2.3s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:   58.0s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [16]:
grid.best_params_

{'C': 100, 'gamma': 0.001}

In [17]:
grid.best_estimator_

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [18]:
pred = grid.predict(X_test)
accuracy_score(y_test,pred)

0.9788401253918495

#### Kernel poly

In [ ]:
param_grid = {'C':[0.1,1,10,100],'gamma':[0.1,0.01,0.001,0.0001],'degree':[0,1]}

grid_poly = GridSearchCV(SVC(kernel='poly'),param_grid,verbose=3,n_jobs=2)
grid_poly.fit(X_train,y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] C=0.1, gamma=0.1, degree=0 ......................................
[CV] C=0.1, gamma=0.1, degree=0 ......................................
[CV]  C=0.1, gamma=0.1, degree=0, score=0.5050369375419744, total=   1.2s
[CV] C=0.1, gamma=0.1, degree=0 ......................................
[CV]  C=0.1, gamma=0.1, degree=0, score=0.5050403225806451, total=   1.2s
[CV] C=0.1, gamma=0.01, degree=0 .....................................
[CV]  C=0.1, gamma=0.1, degree=0, score=0.5050403225806451, total=   1.5s
[CV] C=0.1, gamma=0.01, degree=0 .....................................
[CV]  C=0.1, gamma=0.01, degree=0, score=0.5050369375419744, total=   1.5s
[CV] C=0.1, gamma=0.01, degree=0 .....................................
[CV]  C=0.1, gamma=0.01, degree=0, score=0.5050403225806451, total=   1.3s
[CV] C=0.1, gamma=0.001, degree=0 ....................................
[CV]  C=0.1, gamma=0.01, degree=0, score=0.5050403225806451, total=   

[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  1.3min


[CV]  C=1, gamma=0.01, degree=0, score=0.5050403225806451, total=   1.4s
[CV] C=1, gamma=0.001, degree=0 ......................................
[CV]  C=1, gamma=0.01, degree=0, score=0.5050403225806451, total=   1.3s
[CV] C=1, gamma=0.001, degree=0 ......................................
[CV]  C=1, gamma=0.001, degree=0, score=0.5050369375419744, total=   1.4s
[CV] C=1, gamma=0.001, degree=0 ......................................
[CV]  C=1, gamma=0.001, degree=0, score=0.5050403225806451, total=   1.3s
[CV] C=1, gamma=0.0001, degree=0 .....................................
[CV]  C=1, gamma=0.001, degree=0, score=0.5050403225806451, total=   1.4s
[CV] C=1, gamma=0.0001, degree=0 .....................................
[CV]  C=1, gamma=0.0001, degree=0, score=0.5050369375419744, total=   1.4s
[CV] C=1, gamma=0.0001, degree=0 .....................................
[CV]  C=1, gamma=0.0001, degree=0, score=0.5050403225806451, total=   1.3s
[CV] C=1, gamma=0.1, degree=1 ..........................

[CV] C=100, gamma=0.1, degree=1 ......................................
[CV]  C=100, gamma=0.1, degree=1, score=0.9526209677419355, total=14.2min
[CV] C=100, gamma=0.01, degree=1 .....................................
[CV]  C=100, gamma=0.01, degree=1, score=0.9586971121558092, total= 5.6min
[CV] C=100, gamma=0.01, degree=1 .....................................
[CV]  C=100, gamma=0.01, degree=1, score=0.9566532258064516, total= 6.8min
[CV] C=100, gamma=0.01, degree=1 .....................................
[CV]  C=100, gamma=0.01, degree=1, score=0.9603494623655914, total=10.5min
[CV] C=100, gamma=0.001, degree=1 ....................................
[CV]  C=100, gamma=0.001, degree=1, score=0.9590329079919409, total= 4.2min
[CV] C=100, gamma=0.001, degree=1 ....................................
[CV]  C=100, gamma=0.001, degree=1, score=0.9623655913978495, total= 3.2min
[CV] C=100, gamma=0.001, degree=1 ....................................
[CV]  C=100, gamma=0.001, degree=1, score=0.96034946

In [25]:
grid_poly.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

Não foi possível obter resultado

#### Kernel Sigmoid

In [20]:
param_grid = {'C':[0.1,1,10,100],'gamma':[1,0.1,0.01,0.001,0.0001],'coef0':[-1,0,1]}

grid_sig = GridSearchCV(SVC(kernel='sigmoid'),param_grid,verbose=3,n_jobs=2)
grid_sig.fit(X_train,y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] C=0.1, gamma=1, coef0=-1 ........................................
[CV] C=0.1, gamma=1, coef0=-1 ........................................
[CV]  C=0.1, gamma=1, coef0=-1, score=0.5050369375419744, total=   1.3s
[CV] C=0.1, gamma=1, coef0=-1 ........................................
[CV]  C=0.1, gamma=1, coef0=-1, score=0.5050403225806451, total=   1.3s
[CV] C=0.1, gamma=0.1, coef0=-1 ......................................
[CV]  C=0.1, gamma=0.1, coef0=-1, score=0.5050369375419744, total=   1.3s
[CV] C=0.1, gamma=0.1, coef0=-1 ......................................
[CV]  C=0.1, gamma=1, coef0=-1, score=0.5050403225806451, total=   1.4s
[CV] C=0.1, gamma=0.1, coef0=-1 ......................................
[CV]  C=0.1, gamma=0.1, coef0=-1, score=0.5050403225806451, total=   1.3s
[CV] C=0.1, gamma=0.01, coef0=-1 .....................................
[CV]  C=0.1, gamma=0.1, coef0=-1, score=0.5050403225806451, total=   1.3s
[CV

[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:   34.0s


[CV]  C=0.1, gamma=0.0001, coef0=0, score=0.3897849462365591, total=   2.1s
[CV] C=0.1, gamma=1, coef0=1 .........................................
[CV]  C=0.1, gamma=0.0001, coef0=0, score=0.38676075268817206, total=   3.6s
[CV] C=0.1, gamma=1, coef0=1 .........................................
[CV]  C=0.1, gamma=1, coef0=1, score=0.5050369375419744, total=   1.2s
[CV] C=0.1, gamma=1, coef0=1 .........................................
[CV]  C=0.1, gamma=1, coef0=1, score=0.5050403225806451, total=   1.2s
[CV] C=0.1, gamma=0.1, coef0=1 .......................................
[CV]  C=0.1, gamma=1, coef0=1, score=0.5050403225806451, total=   1.2s
[CV] C=0.1, gamma=0.1, coef0=1 .......................................
[CV]  C=0.1, gamma=0.1, coef0=1, score=0.5050369375419744, total=   1.2s
[CV] C=0.1, gamma=0.1, coef0=1 .......................................
[CV]  C=0.1, gamma=0.1, coef0=1, score=0.5050403225806451, total=   1.3s
[CV] C=0.1, gamma=0.01, coef0=1 ..............................

[CV]  C=1, gamma=0.001, coef0=1, score=0.5050403225806451, total=   1.7s
[CV] C=1, gamma=0.0001, coef0=1 ......................................
[CV]  C=1, gamma=0.001, coef0=1, score=0.5050403225806451, total=   1.8s
[CV] C=1, gamma=0.0001, coef0=1 ......................................
[CV]  C=1, gamma=0.0001, coef0=1, score=0.6430490261920753, total=   2.2s
[CV] C=1, gamma=0.0001, coef0=1 ......................................
[CV]  C=1, gamma=0.0001, coef0=1, score=0.6246639784946236, total=   2.4s
[CV] C=10, gamma=1, coef0=-1 .........................................
[CV]  C=1, gamma=0.0001, coef0=1, score=0.3914650537634409, total=   2.4s
[CV] C=10, gamma=1, coef0=-1 .........................................
[CV]  C=10, gamma=1, coef0=-1, score=0.5050369375419744, total=   1.2s
[CV] C=10, gamma=1, coef0=-1 .........................................
[CV]  C=10, gamma=1, coef0=-1, score=0.5050403225806451, total=   1.4s
[CV] C=10, gamma=0.1, coef0=-1 .................................

[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:  2.4min


[CV]  C=10, gamma=0.1, coef0=1, score=0.5050403225806451, total=   1.3s
[CV] C=10, gamma=0.01, coef0=1 .......................................
[CV]  C=10, gamma=0.1, coef0=1, score=0.5050403225806451, total=   1.2s
[CV] C=10, gamma=0.01, coef0=1 .......................................
[CV]  C=10, gamma=0.01, coef0=1, score=0.5050369375419744, total=   1.3s
[CV] C=10, gamma=0.01, coef0=1 .......................................
[CV]  C=10, gamma=0.01, coef0=1, score=0.5050403225806451, total=   1.3s
[CV] C=10, gamma=0.001, coef0=1 ......................................
[CV]  C=10, gamma=0.01, coef0=1, score=0.5050403225806451, total=   1.2s
[CV] C=10, gamma=0.001, coef0=1 ......................................
[CV]  C=10, gamma=0.001, coef0=1, score=0.5050369375419744, total=   1.9s
[CV] C=10, gamma=0.001, coef0=1 ......................................
[CV]  C=10, gamma=0.001, coef0=1, score=0.5050403225806451, total=   1.8s
[CV] C=10, gamma=0.0001, coef0=1 ..............................

[Parallel(n_jobs=2)]: Done 180 out of 180 | elapsed:  3.4min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'coef0': [-1, 0, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [21]:
grid_sig.best_params_

{'C': 10, 'coef0': 0, 'gamma': 0.0001}

In [22]:
pred = grid_sig.predict(X_test)
accuracy_score(y_test,pred)

0.7494775339602926